<a href="https://colab.research.google.com/github/olonok69/NEO4J/blob/main/NEO4J03_Recommendations_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install py2neo==4.1.3 pandas matplotlib sklearn

     |████████████████████████████████| 71kB 3.2MB/s 
     |████████████████████████████████| 194kB 8.4MB/s 
     |████████████████████████████████| 256kB 20.5MB/s 
     |████████████████████████████████| 133kB 18.5MB/s 
  Created wheel for py2neo: filename=py2neo-4.1.3-py2.py3-none-any.whl size=89805 sha256=48e71b807375d4c4d5d4d3a9f8fdfb8a0e3315ea229bd7ebafd8fa73ea001fff
  Stored in directory: /root/.cache/pip/wheels/49/b4/f8/df22db284f7ec2f6c26f87a87ff38c6c8ffa84baf2930ef0fe
  Created wheel for neo4j-driver: filename=neo4j_driver-1.6.3-cp36-cp36m-linux_x86_64.whl size=525052 sha256=501ee08a897db3af5cf04c345d39a019a4248a46c19add2364472bc37cd2f690
  Stored in directory: /root/.cache/pip/wheels/e3/3d/6f/b1afb54585268913bb38334adaa58c2938911edc85102803a1
  Created wheel for neotime: filename=neotime-1.0.0-cp36-none-any.whl size=14714 sha256=2ddd91e4806da7c699484ab3b5c0158eee0478809644a1a16cbc3e891eca9f58
  Stored in directory: /root/.cache/pip/wheels/19/1c/63/e1777cae4998cb32c5ae5bb1901d

# Recommendations: Part 2

In the 2nd part of our recommendations exercise, you will use the PageRank algorithm to make article recommendations to an author. 
Execute the code to import the libraries (remember to unset Reset all runtimes before running):

In [ ]:
from py2neo import Graph
import pandas as pd

import matplotlib 
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', 100)

Next, create a connection to your Neo4j Sandbox, just as you did previously when you set up your environment. 

<div align="left">
    <img src="https://github.com/neo4j-contrib/training-v2/blob/master/Courses/DataScience/notebooks/images/sandbox-citations.png?raw=1" alt="Citation Sandbox"/>
</div>

Update the cell below to use the IP Address, Bolt Port, and Password, as you did previously.

In [ ]:
# Change the line of code below to use the IP Address, Bolt Port, and Password of your Sandbox.
# graph = Graph("bolt://<IP Address>:<Bolt Port>", auth=("neo4j", "<Password>")) 
 
from py2neo import Graph 
graph = Graph("bolt://100.26.229.35:34820", auth=("neo4j", "dares-problems-concurrence"))

## PageRank

PageRank is an algorithm that measures the transitive influence or connectivity of nodes. It can be computed by either iteratively distributing one node’s rank (originally based on degree) over its neighbors or by randomly traversing the graph and counting the frequency of hitting each node during these walks.

Run this PageRank code over the whole graph to find out the most influential article in terms of citations:

In [ ]:
query = """
CALL algo.pageRank('Article', 'CITED')
"""
graph.run(query).data()

[{'computeMillis': 69,
  'dampingFactor': 0.85,
  'iterations': 20,
  'loadMillis': 142,
  'nodes': 51956,
  'write': True,
  'writeMillis': 405,
  'writeProperty': 'pagerank'}]

This query stores a 'pagerank' property on each node. Execute this code to view the most influential articles:

query = """
MATCH (a:Article)
RETURN a.title as article,
       a.pagerank as score
ORDER BY score DESC 
LIMIT 10
"""
graph.run(query).to_data_frame()

In [ ]:
query = """ MATCH (a:Article) RETURN a.title as article, a.pagerank as score ORDER BY score DESC LIMIT 10 """ 
graph.run(query).to_data_frame()

,article,score
0,A method for obtaining digital signatures and public-key cryptosystems,93.943
1,Secure communications over insecure channels,79.869
2,Rough sets,25.609
3,An axiomatic basis for computer programming,23.029
4,"Pastry: Scalable, Decentralized Object Location, and Routing for Large-Scale Peer-to-Peer Systems",21.470
5,SCRIBE: The Design of a Large-Scale Event Notification Infrastructure,19.486
6,A field study of the software design process for large systems,19.028
7,Productivity factors and programming environments,18.499
8,Analyzing medium-scale software development,16.453
9,A Calculus of Communicating Systems,15.431


## Personalized PageRank

Personalized PageRank is a variant of PageRank that allows us to find influential nodes based on a set of source nodes.

For example, rather than finding the overall most influential articles, we could instead, find the most influential articles with respect to a given author.
Execute this code to use a personalized PageRank algorithm:

In [ ]:
query = """
MATCH (a:Author {name: $author})<-[:AUTHOR]-(article)-[:CITED]->(other)
WITH collect(article) + collect(other) AS sourceNodes
CALL algo.pageRank.stream('Article', 'CITED', {sourceNodes: sourceNodes})
YIELD nodeId, score
RETURN algo.getNodeById(nodeId).title AS article, score
ORDER BY score DESC
LIMIT 10
"""

author_name = "Peter G. Neumann"
graph.run(query, {"author": author_name}).to_data_frame()

,article,score
0,A technique for software module specification with examples,0.359
1,A messy state of the union: taming the composite state machines of TLS,0.332
2,Crypto policy perspectives,0.278
3,Risks of automation: a cautionary total-system perspective of our cyberfuture,0.278
4,The foresight saga,0.278
5,Risks of e-voting,0.278
6,Public interest and the NII,0.278
7,Password security: a case history,0.278
8,The challenges of partially automated driving,0.268
9,Proof techniques for hierarchically structured programs,0.248


## Topic Sensitive Search

You can also use Personalized PageRank to do 'Topic Specific PageRank'. 

When an author is searching for articles to read, they want that search to take themselves as authors into account. Two authors using the same search term would expect to see different results depending on their area of research.

Create a full text search index on the 'title' and 'abstract' properties of all nodes that have the label 'Article' by executing this code:

In [ ]:
query = """
    CALL db.index.fulltext.createNodeIndex('articles', ['Article'], ['title', 'abstract'])
"""
graph.run(query).data()

[]

Check that the full text index has been created by running the following query:

In [ ]:
query = """
CALL db.indexes()
YIELD description, indexName, tokenNames, properties, state, type, progress
WHERE type = "node_fulltext"
RETURN *
"""
graph.run(query).to_data_frame()

,description,indexName,progress,properties,state,tokenNames,type
0,"INDEX ON NODE:Article(title, abstract)",articles,100.000,"[title, abstract]",ONLINE,[Article],node_fulltext


You can search the full text index like this:

In [ ]:
query = """
CALL db.index.fulltext.queryNodes("articles", "open source")
YIELD node, score
RETURN node.title, score, [(author)<-[:AUTHOR]-(node) | author.name] AS authors
LIMIT 10
"""
graph.run(query).to_data_frame()

,node.title,score,authors
0,Progressive open source,4.252,"[Rob Miller, Dean Nelson, Pankaj K. Garg, Jamie Dinkelacker]"
1,Open source application spaces: the 5th workshop on open source software engineering,4.081,"[Walt Scacchi, Joseph Feller, Brian Fitzgerald, Krishna K Lakhani, Scott A. Hissam]"
2,Reusing Open-Source Software and Practices: The Impact of Open-Source on Commercial Vendors,4.071,"[Alan W. Brown, Grady Booch]"
3,Software architecture in an open source world,3.815,[Roy T. Fielding]
4,From Research Software to Open Source,3.784,[Susan L. Graham]
5,Open courseware and open source software,3.693,"[Stefan Baldi, Anett Mehler-Bicher, Hauke Heier]"
6,IBM's pragmatic embrace of open source,3.690,[Pamela Samuelson]
7,When to release in open source project,3.543,"[LiGuo Huang, Zeheng Li]"
8,Increased security through open source,3.515,"[Jaap-Henk Hoepman, Bart Jacobs]"
9,Robust open-source software,3.492,[Peter G. Neumann]


Here is a query to find the authors that have published the most articles on 'open source':

In [ ]:
query = """
CALL db.index.fulltext.queryNodes("articles", "open source")
YIELD node, score
MATCH (node)-[:AUTHOR]->(author)
RETURN author.name, sum(score) AS totalScore, collect(node.title) AS articles
ORDER By totalScore DESC
LIMIT 20
"""

graph.run(query).to_data_frame()

,author.name,totalScore,articles
0,Brian Fitzgerald,16.119,"[Open source application spaces: the 5th workshop on open source software engineering, The 3rd w..."
1,Joseph Feller,16.012,"[Open source application spaces: the 5th workshop on open source software engineering, The 3rd w..."
2,Walt Scacchi,10.731,"[Open source application spaces: the 5th workshop on open source software engineering, The futur..."
3,Daniel M. German,10.687,"[Open source-style collaborative development practices in commercial projects using GitHub, Mach..."
4,Scott A. Hissam,10.642,"[Open source application spaces: the 5th workshop on open source software engineering, The 3rd w..."
5,James D. Herbsleb,10.476,"[A case study of a corporate open source development model, Managing a corporate open source sof..."
6,Denys Poshyvanyk,8.907,"[Machine learning-based detection of open source license exceptions, Recommending source code fo..."
7,Margaret-Anne D. Storey,8.181,"[Understanding broadcast based peer review on open source software projects, Peer Review on Open..."
8,Peter C. Rigby,7.649,"[Understanding broadcast based peer review on open source software projects, Peer Review on Open..."
9,Ruchika Malhotra,7.132,"[An automated tool for generating change report from open-source software, Cross project change ..."


Next, use full text search and Personalized PageRank to find interesting articles for different authors:

In [ ]:
query = """
MATCH (a:Author {name: $author})<-[:AUTHOR]-(article)-[:CITED]->(other)
WITH a, collect(article) + collect(other) AS sourceNodes
CALL algo.pageRank.stream(
  'CALL db.index.fulltext.queryNodes("articles", $searchTerm)
   YIELD node, score
   RETURN id(node) as id',
  'MATCH (a1:Article)-[:CITED]->(a2:Article) 
   RETURN id(a1) as source,id(a2) as target', 
  {sourceNodes: sourceNodes,graph:'cypher', params: {searchTerm: $searchTerm}})
YIELD nodeId, score
WITH algo.getNodeById(nodeId) AS n, score
WHERE not(exists((a)<-[:AUTHOR]-(n))) AND score > 0
RETURN n.title as article, score, [(n)-[:AUTHOR]->(author) | author.name][..5] AS authors
order by score desc limit 10
"""

params = {"author": "Tao Xie", "searchTerm": "open source"}
graph.run(query, params).to_data_frame()

,article,score,authors
0,Static detection of cross-site scripting vulnerabilities,0.386,"[Zhendong Su, Gary Wassermann]"
1,Concern graphs: finding and describing concerns using structural program dependencies,0.278,"[Gail C. Murphy, Martin P. Robillard]"
2,Characterizing logging practices in open-source software,0.278,"[Ding Yuan, Soyeon Park, Yuanyuan Zhou]"
3,"Automated, contract-based user testing of commercial-off-the-shelf components",0.278,"[Lionel C. Briand, Yvan Labiche, Michal M. Sówka]"
4,Who should fix this bug,0.278,"[Lyndon Hiew, John Anvik, Gail C. Murphy]"
5,Conceptual module querying for software reengineering,0.236,"[Gail C. Murphy, Elisa L. A. Baniassad]"
6,Semantics-based code search,0.150,[Steven P. Reiss]
7,Bandera: extracting finite-state models from Java source code,0.150,"[Matthew B. Dwyer, Hongjun Zheng, James C. Corbett, Shawn Laubach, John Hatcliff]"
8,AsDroid: detecting stealthy behaviors in Android applications by user interface and program beha...,0.150,"[Lin Tan, Jianjun Huang, Xiangyu Zhang, Bin Liang, Peng Wang]"
9,EXSYST: search-based GUI testing,0.128,"[Andreas Zeller, Gordon Fraser, Florian Gross]"


Execute the same query with a different author:

In [ ]:
params = {"author": "Marco Aurélio Gerosa", "searchTerm": "open source"}
graph.run(query, params).to_data_frame()

,article,score,authors
0,Toward an understanding of the motivation of open source software developers,0.388,"[Yunwen Ye, Kouichi Kishida]"
1,Hipikat: recommending pertinent software development artifacts,0.322,"[Gail C. Murphy, Davor Cubranic]"
2,Version Sensitive Editing: Change History as a Programming Tool,0.274,[David L. Atkins]
3,Which bug should I fix: helping new developers onboard a new project,0.239,"[Jianguo Wang, Anita Sarma]"
4,Tesseract: Interactive visual exploration of socio-technical relationships in software development,0.203,"[Anita Sarma, Larry Maccherone, Patrick Wagstrom, James D. Herbsleb]"
5,Role Migration and Advancement Processes in OSSD Projects: A Comparative Case Study,0.175,"[Walt Scacchi, Chris Jensen]"
6,Does the initial environment impact the future of developers,0.175,"[Minghui Zhou, Audris Mockus]"
7,Unifying artifacts and activities in a visual tool for distributed software development teams,0.173,"[Jon Froehlich, Paul Dourish]"
8,A case study of open source software development: the Apache server,0.110,"[James D. Herbsleb, Audris Mockus, Roy Fielding]"
9,A case study of the evolution of Jun: an object-oriented open-source 3D multimedia library,0.110,"[Yoshiyuki Nishinaka, Atsushi Aoki, Kouichi Kishida, Y. Yamamoto, Kaoru Hayashi]"
